# **NV Center in Diamond**

In this tutorial we will go over the main steps of running CCE calculations for the NV center in diamond with the **PyCCE** module. Those include:

* Generating the spin bath using the `pycce.BathCell` instance.
* Setting up properties of the `pycce.Simulator` instance.
* Running the calculations with the `Simulator.compute` function.

We will compute the Hahn-echo coherence function (with decoupling $\pi$-pulse) using conventional CCE.

## **(0) Import packages**

In [ ]:
# Version control
!pip install scipy==1.13.0
!pip install numpy==1.26.0

In [ ]:
!pip install pycce
!pip install ase

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sys
import pycce as pc
import ase

from mpl_toolkits import mplot3d

seed = 8805
np.random.seed(seed)
np.set_printoptions(suppress=True, precision=5)

# **=======================================================================**

## **(1) Generate the nuclear spin bath**

There are 4 steps in generating the nuclear spin bath (i.e. the <sup>13</sup>C nuclear spins around the NV center).


### **(1.1) Create BathCell**

We first build the `BathCell` object in PyCCE, which is a precursor to generating the nuclear spin bath.

We can generate the `BathCell` object from an `ase.Atoms` object by using the classmethod `BathCell.from_ase`.

In [ ]:
from ase.build import bulk

# Create diamond unit cell from ase
diamond_ase = bulk('C', 'diamond', cubic=True)  # ASE Atoms object

# Generate PyCCE BathCell
diamond_bc = pc.read_ase(diamond_ase)           # PyCCE BathCell object

The following attributes are created with this initiallization:

* `.cell` is an ndarray containing lattice vector information. Each **column** is a lattice vector in cartesian coordinates.
* `.atoms` is a dictionary with keys corresponding to the atom name, and each item is a list of the coordinates in cell coordinates.

In [ ]:
print('Cell\n', diamond_bc.cell)
print('\nAtoms\n', diamond_bc.atoms)

### **(1.2) Specify isotopes in BathCell**

By default, PyCCE uses the **EasySpin** database (https://easyspin.org/easyspin/documentation/isotopetable.html) containing the concentrations of all common stable isotopes of all elements, however the user can proide custom concentrations.

Use the function `BathCell.add_isotopes` to add one (or several) isotopes of the element. Each isotope is initiallized with a ``tuple`` containing the name of the isotope and its concentration.

Structure of the dictionary-like object:
```
{element_1: {isotope_1: concentration, isotope_2: concentration},
 element_2: {isotope_3: concentration, ...}}
```


In [ ]:
# Add types of isotopes
diamond_bc.add_isotopes(('13C', 0.011))

Isotopes may also be directly added to `BathCell.isotopes`. For example, below we are adding an isotope without a nuclear spin:

In [ ]:
diamond_bc.isotopes['C']['14C'] = 0.001

### **(1.3) Define the quantization axis of the spin defect**

In the `Simulator` object, everything is set in $S_z$ basis. When the quantization axis of the defect does not align with the (0, 0, 1) direction of the crystal axis, the user needs to manually define this axis.

If one wants to specify the complete rotation of Cartesian axes, one can provide a rotation matrix to rotate the Cartesian reference frame with respect to the cell coordinates by calling the `BathCell.rotate` method.

In [ ]:
# set z direction of the defect
diamond_bc.zdir = [1, 1, 1]

### **(1.4) Generate the spin bath**

To generate the spin bath, use the `BathCell.gen_supercell` method.**This is when the NV center is created!**

`sc_size` is the linear size of the supercell in &Aring;.

`remove` takes a ``tuple`` or ``list of tuples`` as an argument. First element of each tuple is the name of the **atom**, and the second element is the fractional coordinate. If such atoms are found in the supercell, they are removed.

`add` takes a ``tuple`` or ``list of tuples`` as an argument. First element of each tuple is the name of the **isotope**, and the second element is the fractional coordinate. Each of the specified isotopes will be added in the final supercell at the specified locations.

In [ ]:
# Add the defect: remove and add atoms at positions (in cell coordinates)
sc_size = 200  # linear size of supercell (A)
diamond_bath = diamond_bc.gen_supercell(sc_size,
                                        remove=[('C', [0., 0, 0]),
                                                ('C', [0.25, 0.25, 0.25])],
                                        add=('14N', [0.5, 0.25, 0.25]),
                                        seed=seed)

Note, that because the `14C` isotope doesn't have a spin, **PyCCE** does not find it in common isotopes, and raises a warning. We have to provide `SpinType` for it separately, or define the properties as follows:

In [ ]:
diamond_bath['14C'].gyro = 0
diamond_bath['14C'].spin = 0

Note that the `BathCell.gen_supercell` method creates a `BathArray` object. This will be discussed in (1.6).

### **(1.5) Visualize the nuclear spin bath**

We can use `matplotlib` to visualize the spatial distribution of the spin bath.

In [ ]:
# add 3D axis
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(projection='3d')

# plot the positions of the bath
ax.scatter3D(diamond_bath.x, diamond_bath.y, diamond_bath.z, c='k')

ax.set(xlabel='x (A)', ylabel='y (A)', zlabel='z (A)');

It looks like a dirty blob; **that's okay**! The `BathCell.gen_supercell` routine simply creates all nuclear spins within the generated supercell (`sc_size`) and with the concentration of each isotope; **the clusters are not created yet**.

Later, we'll define bath parameters `r_bath`, `r_dipole`, and `order`, which will generate the spin bath with clusters.

**Check your understanding**: How many <sup>13</sup>C nuclear spins should we expect to be generated by the `gen_supercell` routine?

### **(1.6) The BathArray structure**

The bath spins are stored in the `BathArray` object - a subclass of `np.ndarray` with fixed datastructure:

* `N` field `dtype('<U16')` contains the names of bath spins.
* `xyz` field `dtype('<f8', (3,))` contains the positions of bath spins (in &Aring;).
* `A` field `dtype('<f8', (3, 3))` contains the hyperfine coupling of bath spins (in kHz).
* `Q` field `dtype('<f8', (3, 3))` contains the quadrupole tensor of bath spins (in kHz) (Relevant for spin &ge; 1).

All of the fields are accesible as attributes of `BathArray`.
Additionally, the subarrays of the specific spins are accessible with their name as indicated above.

Upon generation of the array from the cell, the `Q` and `A` fields are empty. The Hyperfine couplings `A` will be automatically computed by the `Simulator` object, however the quadrupole couplings `Q` must be set by the user.

The additional attributes allow one to access `SpinType` properties:

+ `name` returns the spin name or array of spin names;
+ `spin` returns the value of the spin or array of ones;
+ `gyro` returns gyromagnetic ratios of the spins;
+ `q` returns quadrupole constants of the spins;
+ `h` returns a dictionary with user-defined additions to the Hamiltonian.
+ `detuning` returns detunings of the spins (See definition below).

For example, below we print out the attributes of the first two spins (not all) in the `BathArray`.

In [ ]:
print('Names\n', diamond_bath[:2].N)
print('\nCoordinates\n', diamond_bath[:2].xyz)
print('\nHyperfine tensors\n', diamond_bath[:2].A)
print('\nQuadrupole tensors\n', diamond_bath[:2].Q)

The properties of spin types (gyromagnetic ratio, quadrupole moment, etc.) are stored in the `BathArray.types` attribute, which is an instance of `SpinDict` containing `SpinType` classes. For most known isotopes, `SpinType` can be found in the `pycce.common_isotopes` dictionary, and is set by default (including electrons, which can be set with `N = e`). The user can add additional `SpinType` objects, by calling `BathArray.add_type` method or setting elements of `SpinDict` directly. For details of the first approach see documentation of `SpinDict.add_type` method.

The direct setting of types is rather simple. The user can set elements of `SpinDict` with ``tuple``, containing:

* **(spin, gyromagnetic ratio, quadrupole moment *(optional)*, detuning *(optional)*, )**

  **OR**
  
* **(isotope, spin, gyromagnetic ratio, quadrupole moment *(optional)*, detuning *(optional)*, )**

where:

- **isotope** (*str*) is the name of the given spin (same one as in `N` field of `BathArray`) to define new `SpinType` object.
  The key of `SpinDict` **has** to be the correct name of the spin ("isotope" field in the tuple).
- **spin** (*float*) is the total spin of the given bath spin.
- **gyromagnetic ratio** (*float*) is the gyromagnetic ratio of the given bath spin.
- **quadrupole moment** (*float*) is the quadrupole moment of the given bath spin. Relevant only when electric field gradient are used
  to generate quadrupole couplings for spins, stored in the ``BathArray``, with ``BathArray.from_efg`` method.
- **detuning** (float) is an additional energy splitting for model spins, included as an extra $+\omega \hat S_z$ term in the Hamiltonian,
  where $\omega$ is the detuning.

Units of gyromagnetic ratio are rad / ms / G, quadrupole moments are given in barn, detunings are given in kHz.

In [ ]:
# Several ways to set SpinDict elements
diamond_bath.types['14C'] = 0, 0, 0
diamond_bath.types['Y'] = ('Y', 0, 0, 0)
diamond_bath.types['A'] = pc.SpinType('A', 0, 0, 0)

print(diamond_bath.types)

### **(1.7) *Try it yourself* : V<sub>B</sub><sup>-</sup> in hBN**

Let's try generating a nuclear spin bath representative of the environment around the V<sub>B</sub><sup>-</sup> center in hBN.

Below, we've provided the `ase.Atoms` object for the hBN lattice.

In [ ]:
from ase import Atoms

a = 2.504
c = 6.661

cell = [
    [a, 0, 0],
    [-a/2, np.sqrt(3)*a/2, 0],
    [0, 0, c]
]

symbols = ['B', 'N', 'B', 'N']

scaled_positions = [
    [1/3, 2/3, 1/4],
    [2/3, 1/3, 1/4],
    [2/3, 1/3, 3/4],
    [1/3, 2/3, 3/4]
]

hbn_bulk_ase = Atoms(
    symbols=symbols,
    scaled_positions=scaled_positions,
    cell=cell,
    pbc=True
)

print(hbn_bulk_ase)


In [ ]:
# Begin here


# **=======================================================================**

## **(2) Run a CCE simulation**

Here we describe how to run CCE simulations.

### **(2.1) The `Simulator` class**

The `Simulator` object enables CCE simulations.

Main parameters to consider:

* `spin` — Either an instance of the `CenterArray` object, or float - total spin of the central spin (assuming one central spin).

* `position` — Array of Cartesian coordinates (in &Aring;) of the central spin(s).

* `bath` — spin bath in any specified format. Can be either:

  - Instance of `BathArray` class
  - `ndarray` with ``dtype([('N', np.str_, 16), ('xyz', np.float64, (3,))])`` containing names of bath spins (same ones as stored in self.ntype) and positions of the spins in angstroms
  - The name of the `.xyz` text file containing 4 columns: name of the bath spin and xyz coordinates in &Aring;

* `r_bath` — cutoff radius (in &Aring;) around the central spin for the bath.

* `r_dipole` — cutoff radius (in &Aring;) for the pairwise distance to consider two nuclear spins to be connected.

* `order` — maximum size of nuclear spin clusters.

* `magnetic_field` — applied magnetic field (in Gauss). Can also be provided during the simulation run.

* `pulses` — number of pulses in Carr-Purcell-Meiboom-Gill (CPMG) sequence or the pulse sequence itself.

For the full description see the documentation of the `Simulator` object (https://pycce.readthedocs.io/en/stable/simulator.html).

### **(2.2) Visualizing the nuclear spin clusters:**

Let's set up a "mock" instance of `Simulator` to visualize the smaller part of the bath around the central spin.

In [ ]:
# Setting the runner engine
mock = pc.Simulator(spin=1,
                    position=[0,0,0],
                    bath=diamond_bath,
                    r_bath=20,
                    r_dipole=6,
                    order=3
                    )

During the initiallization, depending on the provided keyword arguments several methods may be called:

* `Simulator.read_bath` is called if keyword `bath` is provided. It may take several additional arguments:

    * `r_bath` - cutoff distance from the qubit for the bath.
    * `skiprows` - if `bath` is provided as `.xyz` file, this argument tells how many rows to skip when reading the file.
    * `external_bath` - `BathArray` instance, which contains bath spins with pre defined hyperfines to be used.
    * `hyperfine` - defines the way to compute hyperfine couplings. If it is not given and `bath` doesn't contain any predefined hyperfines
      (`bath['A'].any() == False`) the point dipole approximation is used.
      Otherwise it can be an instance of ``pc.Cube`` object, or callable with signature ``func(coord, gyro, central_gyro)``,
      where `coord` is an array of the bath spin coordinate, gyro is the gyromagnetic ratio of bath spin,
      central_gyro is the gyromagnetic ratio of the central bath spin.
    * `types` - instance of `SpinDict` or input to create one.
    * `error_range` - maximum allowed distance between positions in `bath` and `external_bath` for two spins to be considered the same.
    * `ext_r_bath` - cutoff distance from the qubit for the `external_bath`.
      Useful if `external_bath` has very assymetric shape and user wants to keep the precision level of the hyperfine at different distances consistent.
    * `imap` - instance of the `pc.InteractionMap` class, which contain tensor of bath spin interactions.
      If not provided, interactions between bath spins are assumed to be the same as one of point dipoles.
   
  Generates `BathArray` object with hyperfine tensors to be used in the calculation.

* `Simulator.generate_clusters` is called if `order` and `r_dipole` are provided. It produces a `dict` object, which contains the indexes of the bath spins in the clusters.

We implemented the following procedure to determine the clusters:
  
  * Each bath spin $i$ forms a **cluster of one**.
  * Bath spins $i$ and $j$ form a **cluster of two** if there is an edge between them (distance $d_{ij} \le$ `r_dipole`).
  * Bath spins $i$, $j$, and $k$ form a **cluster of three** if enough edges connect them (e.g., there are two edges $ij$ and $jk$).
  
In general, we assume that spins $\{1..n\}$ form clusters if they form a connected graph. Only clusters up to the size indicated by the `order` parameter (equal to the **CCE order**) are included.


We use `matplotlib` to visualize the spatial distribution of the spin bath. The grey lines show connected pairs of nuclear spins, red dashed lines show clusters of three.

In [ ]:
# add 3D axis
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(projection='3d')

# We want to visualize the smaller bath
data = mock.bath

# First plot the positions of the bath
colors = np.abs(data.A[:,2,2]/data.A[:,2,2].max())
ax.scatter3D(data.x, data.y, data.z, c=colors, cmap='viridis');

# Plot all pairs of nuclear spins, which are contained
# in the calc.clusters dictionary under they key 2
for c in mock.clusters[2]:
    ax.plot3D(data.x[c], data.y[c], data.z[c], color='grey')

# Plot all triplets of nuclear spins, which are contained
# in the calc.clusters dictionary under they key 3
for c in mock.clusters[3]:
    ax.plot3D(data.x[c], data.y[c], data.z[c], color='red', ls='--', lw=0.5)

ax.set(xlabel='x (A)', ylabel='y (A)', zlabel='z (A)');

Adjust the `r_dipole`, `r_bath`, or `order` parameters and visualize the spin bath.

### **(2.2.1) *Try it yourself* : V<sub>B</sub><sup>-</sup> in hBN**

Generate the spin bath clusters around the V<sub>B</sub><sup>-</sup> center in hBN. Complete (1.6) if you haven't already.

You should see more nuclear spins and more clusters in this case (**Why?**).

In [ ]:
# Begin here


### **(2.3) Setting up a CCE simulation**

Now let's set up the `Simulator` object for a CCE simulation.

In [ ]:
# Parameters of CCE calculations engine

# Order of CCE aproximation
order = 2  # singlets and pairs of nuclear spins

# Bath cutoff radius
r_bath = 40  # in A

# Cluster cutoff radius
r_dipole = 8  # in A


We will use the ``CenterArray`` object to store the properties of the central spin.

In [ ]:
# position of central spin
position = [0, 0, 0]

# Qubit levels (in Sz basis)
alpha = [0, 0, 1]; beta = [0, 1, 0]

# ZFS Parametters of NV center in diamond
D = 2.88 * 1e6 # in kHz
E = 0 # in kHz
nv = pc.CenterArray(spin=1, position=position, D=D, E=E, alpha=alpha, beta=beta)

**Important**: The position of the central spin should match the position of the NV center you created in (1.4)!

The code already knows the properties of the most common nuclear spins and of the elecron spin (accessible under the name `'e'`), however the user can provide their own by calling `BathArray.add_type` method. The way to initiallize `SpinType` objects is the same as in  `SpinDict` above.

In [ ]:
# The code already knows most exsiting isotopes.
#              Bath spin types
#              name    spin    gyro       quadrupole (for s>1/2)
spin_types = [('14N',  1,      1.9338,    20.44),
              ('13C',  1 / 2,  6.72828),
              ('29Si', 1 / 2, -5.3188),]
diamond_bath.add_type(*spin_types)

Now we can set up the `Simulator` object.

In [ ]:
# Setting the runner engine
calc = pc.Simulator(spin=nv, bath=diamond_bath,
                    r_bath=r_bath, r_dipole=r_dipole, order=order)

Taking advantage of subclassing `np.ndarray` we can change the hyperfine and quadrupole tensors of any nuclear spin. Here, we update the quadrupole tensor of the Nitrogen nuclear spin.

In [ ]:
# Set model quadrupole tensor at N atom
quad = np.asarray([[-2.5, 0, 0],
                   [0, -2.5, 0],
                   [0, 0,  5.0]]) * 1e3 * 2 * np.pi

calc.bath['Q'][calc.bath['N'] == '14N'] = quad


Note, that we need to apply the boolean mask **second** because of how structured arrays work.

### **(2.4) Compute the coherence function (conventional CCE)**

The general interface to compute any property with PyCCE is implemented through the `Simulator.compute` method. It takes two keyword arguments to determine which quantity to compute and how:

* `method` can take 'cce' or 'gcce' values, and determines which method to use - conventional or generalized CCE.
* `quantity` can take 'coherence' or 'noise' values, and determines which quantity to compute - coherence function
  or autocorrelation function of the noise.

Each of the methods can be performed with Monte Carlo bath state sampling (if `nbstates` keyword is non zero) and with interlaced averaging (If `interlaced` keyword is set to `True`).

In the first example we use the conventional CCE method without Monte Carlo bath state sampling. In the conventional CCE method the Hamiltonian is projected on the qubit levels, and the coherence is computed from the overlap of the bath evolution, entangled with two different qubit states.

The conventional CCE requires one argument:

* `timespace` — time points at which the coherence function is computed.

Additionally, one can provide the following arguments now, instead of when initializing ``Simulator`` object:

* `pulses` — number of pulses in CPMG sequence (0 - free induction decay, 1 - Hahn echo, etc., default 0) or explicit sequence of pulses as `Sequence` class instance.
* `magnetic_field` — magnetic field along z-axis or vector of the magnetic field. Default (0, 0, 0).

In [ ]:
# Time points
time_space = np.linspace(0, 2, 1001)  # in ms

# Number of pulses in CPMG seq (0 = FID, 1 = HE)
n = 1

# Magnetic field (Bx By Bz)
b = np.array([0, 0, 15])  # in G

# Compute coherence function
l_conv = calc.compute(time_space, pulses=n, magnetic_field=b,
                      method='cce', quantity='coherence', as_delay=False)

Let's plot the coherence function.

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

ax.plot(time_space, l_conv)
ax.set(xlabel='Time (ms)', ylabel='Coherence')

There are two features of interest here:

1.   The wiggles. These oscillations are caused by the precession of <sup>13</sup>C nuclear spins in a magnetic field.
2.   The overall decay. The decay is caused by flip-flop dynamics of <sup>13</sup>C pairs.

For more information, check out [this paper](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.85.115303) on decoherence dynamics in different magnetic field regimes.


### **(2.5)** Fit $T_2$ and $n$

We can also fit the coherence function to a stretched exponential function,

$$
\mathcal{L}(t) = \exp\left(- \left(\frac{t}{T_2}\right)^n \right)
$$

to obtain the spin-echo coherence time ($T_2$) and the stretch exponential ($n$).

**Note**: Because we computed the coherence function by applying a single pulse `n=1`, we fit $T_2$ (the Hahn-echo coherence time).

In [ ]:
from scipy.optimize import curve_fit

# Stretched exponential fitting function
def Lfunc(time, T2, n):
  return np.exp(-(time/T2)**n)

# Fit T2 and n
param, conv = curve_fit(Lfunc, time_space, l_conv)

print(f'T2: {round(param[0], 2)} ms')
print(f'n: {round(param[1], 2)}')

We can check the goodness of our fit.

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

ax.plot(time_space, l_conv, label="CCE")
ax.plot(time_space, Lfunc(time_space, *param), label="Fit")

ax.set(xlabel='Time (ms)', ylabel='Coherence')
ax.legend()

### **(2.6) Check for convergence**

It is important to check the convergence of the CCE simulation with respect to the `order`, `r_bath`, and `r_dipole` parameters of the `Simulator` object.

First, define all of the parameters.

In [ ]:
parameters = dict(
    order          = 2,           # CCE order
    r_bath         = 40,          # Size of the bath in A
    r_dipole       = 8,           # Cutoff of pairwise clusters in A
    position       = [0, 0, 0],   # Position of central Spin
    alpha          = [0, 0, 1],
    beta           = [0, 1, 0],
    pulses         = 1,           # N pulses in CPMG sequence
    magnetic_field = [0, 0, 500]
)

time_space = np.linspace(0, 2, 201)  # Time points in ms

We can define a little helper function to streamline the process. Note that resetting the parameters automatically recomputes the properties of the bath.

In [ ]:
def runner(variable, values):
    invalue = parameters[variable]
    calc = pc.Simulator(spin=1, bath=diamond_bath, **parameters)
    lcoh = []
    for v in values:
        setattr(calc, variable, v)
        l = calc.compute(time_space, method='cce',
                         quantity='coherence')
        lcoh.append(l.real)
    parameters[variable] = invalue
    df_coh = pd.DataFrame(lcoh, columns=time_space, index=values).T
    return df_coh

Now we can compute the coherence function at different values of the parameters:

In [ ]:
orders = runner('order', [1, 2, 3, 4])
rbs = runner('r_bath', [20, 30, 40, 50, 60])
rds = runner('r_dipole', [4, 6, 8, 10])

We can visualize the convergence of the coherence function with respect to different parameters:

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3))
orders.plot(ax=axes[0], title='order')
rbs.plot(ax=axes[1], title='r_bath')
rds.plot(ax=axes[2], title='r_dipole')
for ax in axes:
    ax.set(xlabel='Time (ms)', ylabel='Coherence')
fig.tight_layout()